In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, json
from urllib import request, parse
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import requests
from time import sleep
import time, timeit
from selenium.webdriver.common.keys import Keys
# 포지션 
from skimage import io # 미니맵 처리
from sklearn.preprocessing import MinMaxScaler #스케일 재조정.


In [2]:
#챔피언 id/이름 목록 파일 불러오기
with open('champion_id_and_name_list.pickle','rb') as f1:
    champ_id_dict=pickle.load(f1)

In [110]:
def parse_tup(summonerName, champion, recent=False, champW=False): #동원님 스크립트를 이용해서 함수 재조합.
    global driver, champ_id_dict
    name_parsed = parse.quote_plus(summonerName)
    if str(champion).isnumeric():
        champ = champ_id_dict[champion] #id -챔피언명 정보 딕셔너리로 불러와주세요. 딕셔너리 이름 다르게 했으면 이 이름으로 하셔야 합니다. 
    else: #영어면 
        champ = champion
    driver.get(f'https://www.op.gg/summoner/userName={name_parsed}') #url 오류
    if not(recent):
        try:
            
            driver.find_element_by_xpath('//*[@id="right_gametype_soloranked"]/a').click()
            sleep(2)
            try:
                recentW_R = int(driver.find_element_by_css_selector('div.WinRatioGraph>div.Text').get_attribute('innerText').replace('%',""))
            except:
                recentW_R = -1
            try:
                recentKDA= float(driver.find_element_by_css_selector('span.KDARatio').get_attribute('innerText').split(':')[0])
            except:
                recentKDA = -1
            try:
                totalW_R=int(driver.find_element_by_css_selector('span.winratio').get_attribute('innerText').split()[-1][:-1])
            except:
                totalW_R = -1
        except:
            recentW_R = 0
            recentKDA = 0
            totalW_R = 0
    else: #recent 데이터가 있으면
        recentW_R = recent[0]
        recentKDA = recent[1]
        totalW_R = recent[2]
    sleep(2)
    
    sleep(2)
    driver.find_element_by_xpath('//*[@id="left_champion"]/a').click()
    sleep(3)
    driver.find_element_by_xpath('//*[@id="champion_season"]/li[2]/a').click()
    sleep(5)
    tables = driver.find_elements_by_tag_name('table')
    trs=tables[-1].find_elements_by_xpath('./tbody/tr')
    #픽 champion의 시즌 승률 검색 만약 데이터가 없으면 최근승률로 대체
    if not(champW):
        for tr in trs:
            tds = tr.find_elements_by_xpath('./td')
            if tds[2].get_attribute('data-value')==champ:
                try:
                    chamW_R=round(float(tds[3].get_attribute('data-value')))
                except:
                    chamW_R = -1
                try:
                    chamKDA=float(tds[4].get_attribute("data-value"))
                except:
                    chamKDA = -1
                try:
                    try:
                        matchCNT1=int(tds[3].find_element_by_css_selector('div.Text.Left').get_attribute('innerText')[:-1])
                    except:
                        matchCNT1 = 0
                    try:
                        matchCNT2 = int(tds[3].find_element_by_css_selector('div.Text.Right').get_attribute('innerText')[:-1])
                    except:
                        matchCNT2 = 0
                    matchCNT = matchCNT1 + matchCNT2
                except: #안 되면 Xpath 방식으로 긁어와보자.
                    try:
                        matchCNT1 = int(tds[3].find_element_by_xpath(',/div/div/div[2]').get_attribute('innerText')[:-1])
                        try:
                            matchCNT2 = int(tds[3].find_element_by_xpath(',/div/div/div[4]').get_attribute('innerText')[:-1])
                        except:
                            matchCNT2 = 0
                        matchCNT = matchCNT1 + matchCNT2
                    except:
                        matchCNT = -1
                break
        else:
            chamW_R=recentW_R
            chamKDA=recentKDA
            matchCNT = 0
        sleep(2)
    else:
        chamW_R = champW[0]
        chamKDA = champW[1]
        matchCNT = champW[2]
        

    return [summonerName,champ,chamW_R,chamKDA,recentW_R,recentKDA, totalW_R ,matchCNT]
    
    

In [115]:
#잘 작동하는 지 확인하기
driver= webdriver.Chrome() #이번엔 옵션 없애기...
driver.get('https://op.gg/')
driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
sleep(2)
lang_list = driver.find_elements_by_class_name('setting-list__item')
for lang in lang_list:
    if "English" in lang.text:
        lang.click()
sleep(2)
driver.find_elements_by_class_name('setting__button')[0].click()
sleep(2)

print(parse_tup('이즈아펠케틀애쉬', 'Aphelios', False, False))
driver.quit()

['이즈아펠케틀애쉬', 'Aphelios', 53, 2.63, 45, 2.37, 50, 284]


In [118]:
dat_result = [] #GameInfo with user 파일 호출
with open('Gameinfo_with_user_48336_to_60000_all.pickle', 'rb') as f:
    dat_result += pickle.load(f)

In [119]:
driver= webdriver.Chrome() #우선 영어로 지정.
driver.get('https://op.gg/')
driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
sleep(2)
lang_list = driver.find_elements_by_class_name('setting-list__item')
for lang in lang_list:
    if "English" in lang.text:
        lang.click()
sleep(2)
driver.find_elements_by_class_name('setting__button')[0].click()
sleep(2)

#gameinfo with user의 데이터 중 chamW_R, chamKDA, totalNUm 보강
for dat in dat_result:
    sleep(2)
    participants = dat['participants'] # participant data 가져오기
    for j in range(len(participants)): #길이가 
        part_data = participants[j]
        print(part_data)
        if part_data[-1] == 0:
            champW = False
        else:
            champW = part_data[-6:-4]+part_data[-1:]
        if champW == False:
            try: #없을 수도 있으니 
                part_data = parse_tup(part_data[0], part_data[1], part_data[-4:-1], champW)
                participants[j] = part_data
            except:
                continue
        sleep(1)
    dat['participants'] = participants
    print(participants)

driver.quit()
    

['뱅 배준식', 'Camille', 50, 1.95, 57, 2.52, 51, 12]
['짜 룽', 'Lee Sin', 51, 3.56, 57, 2.5, 51, 632]
['happy   game ', 'Yone', 48, 1.34, 48, 1.34, 52, 0]
['잠 이 덜 깨다', 'Caitlyn', 44, 2.2, 44, 2.2, 51, 0]
['ha1 m1an bao bao', 'Taric', 61, 3.95, 50, 1.72, 54, 341]
[['뱅 배준식', 'Camille', 50, 1.95, 57, 2.52, 51, 12], ['짜 룽', 'Lee Sin', 51, 3.56, 57, 2.5, 51, 632], ['happy   game ', 'Yone', 40, 1.62, 48, 1.34, 52, 10], ['잠 이 덜 깨다', 'Caitlyn', 57, 2.4, 44, 2.2, 51, 93], ['ha1 m1an bao bao', 'Taric', 61, 3.95, 50, 1.72, 54, 341]]
['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52]
['라인양보좀부캐임', 'Lee Sin', 25, 2.26, 44, 2.09, 52, 12]
['향 유', 'Wukong', 54, 2.24, 48, 2.53, 50, 24]
['날씨요정 정현', 'Ashe', 51, 2.79, 52, 3.16, 51, 59]
['Navy SEAL 이근', 'Pantheon', 75, 2.92, 44, 1.85, 53, 40]
[['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52], ['라인양보좀부캐임', 'Lee Sin', 25, 2.26, 44, 2.09, 52, 12], ['향 유', 'Wukong', 54, 2.24, 48, 2.53, 50, 24], ['날씨요정 정현', 'Ashe', 51, 2.79, 52, 3.16, 51, 59], ['Navy SEAL 이근', 'Pantheon', 75, 2

['bot no', 'Lucian', 47, 2.71, 50, 3.95, 51, 91]
['작은꼬마여우', 'Thresh', 58, 3.1, 59, 3.99, 53, 113]
[['넌너무감정적이야', 'Orianna', 53, 3.13, 41, 2.43, 50, 66], ['최함몰', 'Sylas', 63, 2.51, 41, 2.76, 57, 51], ['토리초리', 'Kayn', 65, 3.47, 53, 2.7, 52, 23], ['bot no', 'Lucian', 47, 2.71, 50, 3.95, 51, 91], ['작은꼬마여우', 'Thresh', 58, 3.1, 59, 3.99, 53, 113]]
['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40]
['토이 콤플렉스', 'Pyke', 50, 2.77, 53, 3.21, 51, 175]
['김해 마이', 'Master Yi', 54, 2.76, 45, 2.11, 51, 677]
['종 서', 'Jhin', 54, 2.6, 46, 1.43, 52, 50]
['Loneliest Girl', 'Sylas', 38, 1.49, 34, 1.73, 51, 108]
[['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40], ['토이 콤플렉스', 'Pyke', 50, 2.77, 53, 3.21, 51, 175], ['김해 마이', 'Master Yi', 54, 2.76, 45, 2.11, 51, 677], ['종 서', 'Jhin', 54, 2.6, 46, 1.43, 52, 50], ['Loneliest Girl', 'Sylas', 38, 1.49, 34, 1.73, 51, 108]]
['Alphabet N', 'Gangplank', 55, 2.87, 42, 1.74, 49, 163]
['adad00ad', 'Orianna', 47, 3.26, 47, 3.26, 50, 0]
['jungukb', 'Lee Sin', 34, 2.1, 56, 1.87, 49,

['Loneliest Girl', 'Jax', 58, 2.24, 34, 1.73, 51, 1490]
['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40]
['청순한 주황버섯', 'Lux', 59, 3.61, 55, 2.86, 52, 54]
['Nigror', 'Lillia', 54, 3.52, 60, 2.59, 50, 74]
['Traho', 'Caitlyn', 58, 3.33, 52, 3.75, 51, 113]
[['Loneliest Girl', 'Jax', 58, 2.24, 34, 1.73, 51, 1490], ['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40], ['청순한 주황버섯', 'Lux', 59, 3.61, 55, 2.86, 52, 54], ['Nigror', 'Lillia', 54, 3.52, 60, 2.59, 50, 74], ['Traho', 'Caitlyn', 58, 3.33, 52, 3.75, 51, 113]]
['yuyuseong', 'Morgana', 62, 4.29, 62, 4.29, 51, 0]
['Lv 1 세구리', 'Camille', 50, 2.14, 47, 2.83, 54, 74]
['원딜이죽지않아', 'Twisted Fate', 45, 3.21, 45, 3.21, 51, 0]
['정신병있는베인', 'Vayne', 54, 1.89, 39, 2.38, 54, 231]
['god호신', 'Nidalee', 35, 3.11, 59, 2.32, 55, 17]
[['yuyuseong', 'Morgana', 49, 2.36, 62, 4.29, 51, 274], ['Lv 1 세구리', 'Camille', 50, 2.14, 47, 2.83, 54, 74], ['원딜이죽지않아', 'Twisted Fate', 33, 3.75, 45, 3.21, 51, 3], ['정신병있는베인', 'Vayne', 54, 1.89, 39, 2.38, 54, 231], ['god호신', 'Nidalee

['DRX 080', 'Jhin', 44, 2.01, 44, 2.01, 62, 0]
[['숨결을 만지던 밤', 'Akali', 68, 2.34, 46, 1.8, 61, 22], ['섹시한 핫세', 'Irelia', 52, 1.74, 54, 3.91, 50, 33], ['룰루 바드 잔나', 'Elise', 67, 2.15, 45, 1.99, 49, 3], ['에밀리 다이어', 'Seraphine', 60, 2.33, 50, 2.95, 50, 5], ['DRX 080', 'Jhin', 44, 2.01, 44, 2.01, 62, 0]]
['ADout CarRy', 'Lucian', 43, 2.43, 43, 2.43, 49, 0]
['단검이 어디있지', 'Camille', 44, 2.46, 44, 2.46, 52, 0]
['안전밸트꼭매라', 'Rammus', 52, 2.34, 52, 2.34, 51, 0]
['메라병 그랩', 'Nami', 53, 4.26, 53, 4.26, 70, 0]
['하은이 미드', 'Anivia', 49, 2.21, 49, 2.21, 52, 0]
[['ADout CarRy', 'Lucian', 40, 1.98, 43, 2.43, 49, 30], ['단검이 어디있지', 'Camille', 44, 2.46, 44, 2.46, 52, 0], ['안전밸트꼭매라', 'Rammus', 52, 2.34, 52, 2.34, 51, 0], ['메라병 그랩', 'Nami', 53, 4.26, 53, 4.26, 70, 0], ['하은이 미드', 'Anivia', 49, 2.21, 49, 2.21, 52, 0]]
['AfreecaTV 이인수', 'LeBlanc', 67, 2.62, 51, 2.53, 52, 9]
['살려주시오제발', 'Kindred', 50, 3.08, 53, 3.66, 55, 2]
['우한폐렴인데요', 'Ashe', 50, 2.16, 45, 1.82, 50, 4]
['전략적버러지', 'Seraphine', 50, 4.15, 50, 4.15, 60

['더샤이3', 'Sylas', 36, 2.0, 45, 2.29, 35, 11]
['v와드싸개v', 'Senna', 55, 3.13, 46, 2.0, 45, 22]
['철신지옥의이꾸', 'Amumu', 50, 1.92, 50, 1.92, 43, 0]
['코칼리', 'Aphelios', 43, 2.14, 57, 2.91, 52, 14]
['빠가라', 'Garen', 50, 3.2, 50, 3.2, 52, 0]
[['더샤이3', 'Sylas', 36, 2.0, 45, 2.29, 35, 11], ['v와드싸개v', 'Senna', 55, 3.13, 46, 2.0, 45, 22], ['철신지옥의이꾸', 'Amumu', 50, 1.92, 50, 1.92, 43, 0], ['코칼리', 'Aphelios', 43, 2.14, 57, 2.91, 52, 14], ['빠가라', 'Garen', 58, 3.56, 50, 3.2, 52, 52]]
['adventure hunter', 'Kindred', 57, 2.6, 52, 2.1, 54, 556]
['엄딩이 조아', 'Volibear', 25, 1.4, 48, 3.06, 56, 12]
['한계일때 한개 더', 'Zoe', 57, 3.67, 46, 2.06, 52, 49]
['Sniper Gun', 'Pantheon', 20, 1.83, 43, 1.97, 52, 5]
['Ptolemaeos', 'Jhin', 51, 2.39, 52, 2.23, 51, 84]
[['adventure hunter', 'Kindred', 57, 2.6, 52, 2.1, 54, 556], ['엄딩이 조아', 'Volibear', 25, 1.4, 48, 3.06, 56, 12], ['한계일때 한개 더', 'Zoe', 57, 3.67, 46, 2.06, 52, 49], ['Sniper Gun', 'Pantheon', 20, 1.83, 43, 1.97, 52, 5], ['Ptolemaeos', 'Jhin', 51, 2.39, 52, 2.23, 51, 84]]


...

['축구선수 루시안', 'Pantheon', 49, 5.11, 49, 5.11, 67, 0]
['싱하횽', 'Yasuo', 51, 2.69, 51, 2.69, 47, 0]
['지니야음악틀어줘', 'Jhin', 52, 3.34, 52, 3.34, 48, 0]
['똥주 뿌신쩌우', 'Malphite', 40, 1.73, 33, 2.1, 56, 5]
['서랜안치는놈', 'Fiddlesticks', 51, 2.21, 51, 2.21, 50, 0]
[['축구선수 루시안', 'Pantheon', 100, 3.33, 49, 5.11, 67, 2], ['싱하횽', 'Yasuo', 54, 2.17, 51, 2.69, 47, 57], ['지니야음악틀어줘', 'Jhin', 100, 6.11, 52, 3.34, 48, 4], ['똥주 뿌신쩌우', 'Malphite', 40, 1.73, 33, 2.1, 56, 5], ['서랜안치는놈', 'Fiddlesticks', 55, 2.45, 51, 2.21, 50, 11]]
['송효승', 'Shen', 35, 2.01, 35, 2.01, 50, 0]
['어둠의 왕왕', 'Samira', 59, 2.54, 52, 2.14, 56, 22]
['힘내세요 화이띵', 'Morgana', 54, 1.71, 54, 1.71, 53, 0]
['DEBOO', 'Lee Sin', 51, 2.84, 52, 3.16, 49, 41]
['그마가면 공부함', 'Galio', 45, 3.07, 45, 3.07, 53, 0]
[['송효승', 'Shen', 60, 3.06, 35, 2.01, 50, 5], ['어둠의 왕왕', 'Samira', 59, 2.54, 52, 2.14, 56, 22], ['힘내세요 화이띵', 'Morgana', 51, 2.94, 54, 1.71, 53, 43], ['DEBOO', 'Lee Sin', 51, 2.84, 52, 3.16, 49, 41], ['그마가면 공부함', 'Galio', 62, 4.31, 45, 3.07, 53, 37]]
['MO

['OZ Crown', 'Twisted Fate', 56, 2.34, 43, 1.44, 50, 126]
['OZ MaHa', 'Miss Fortune', 51, 2.99, 55, 2.92, 54, 164]
['Twitch 도위', 'Lee Sin', 54, 2.66, 51, 1.68, 52, 177]
['Fall Guy', 'Leona', 56, 2.43, 47, 2.13, 54, 318]
['팬더는 짹짹', 'Vladimir', 57, 2.01, 36, 1.64, 53, 580]
[['OZ Crown', 'Twisted Fate', 56, 2.34, 43, 1.44, 50, 126], ['OZ MaHa', 'Miss Fortune', 51, 2.99, 55, 2.92, 54, 164], ['Twitch 도위', 'Lee Sin', 54, 2.66, 51, 1.68, 52, 177], ['Fall Guy', 'Leona', 56, 2.43, 47, 2.13, 54, 318], ['팬더는 짹짹', 'Vladimir', 57, 2.01, 36, 1.64, 53, 580]]
['OZ Crown', 'Twisted Fate', 56, 2.34, 43, 1.44, 50, 126]
['OZ MaHa', 'Miss Fortune', 51, 2.99, 55, 2.92, 54, 164]
['해삐 어네삐', 'Janna', 75, 5.5, 58, 4.56, 63, 4]
['묵언수행 스님1', 'Lee Sin', 39, 1.6, 39, 1.6, 50, 0]
['협곡의개', 'Renekton', 40, 1.92, 40, 1.92, 53, 0]
[['OZ Crown', 'Twisted Fate', 56, 2.34, 43, 1.44, 50, 126], ['OZ MaHa', 'Miss Fortune', 51, 2.99, 55, 2.92, 54, 164], ['해삐 어네삐', 'Janna', 75, 5.5, 58, 4.56, 63, 4], ['묵언수행 스님1', 'Lee Sin', 47,

['돌려줘', 'Camille', 38, 1.74, 38, 1.74, 55, 0]
['Flaycid', 'Vayne', 70, 3.21, 47, 1.61, 53, 10]
['Cor0navirus', 'Shen', 60, 3.4, 41, 1.38, 55, 197]
[['박보영 이뽀영', 'Orianna', 50, 3.59, 52, 2.96, 49, 248], ['빨리못참겟어', 'Jarvan IV', 49, 3.52, 62, 3.39, 49, 462], ['돌려줘', 'Camille', 59, 2.24, 38, 1.74, 55, 68], ['Flaycid', 'Vayne', 70, 3.21, 47, 1.61, 53, 10], ['Cor0navirus', 'Shen', 60, 3.4, 41, 1.38, 55, 197]]
['짤서폿만하는사람', 'Xerath', 53, 2.9, 50, 2.8, 51, 529]
['kdlvfmq', 'Ezreal', 54, 2.7, 48, 2.21, 52, 194]
['JaxCalibur', 'Wukong', 49, 1.95, 44, 2.08, 51, 63]
['OpheliaKR', 'Twisted Fate', 55, 1.94, 52, 1.94, 51, 11]
['await0', 'Lee Sin', 48, 2.76, 54, 2.46, 53, 21]
[['짤서폿만하는사람', 'Xerath', 53, 2.9, 50, 2.8, 51, 529], ['kdlvfmq', 'Ezreal', 54, 2.7, 48, 2.21, 52, 194], ['JaxCalibur', 'Wukong', 49, 1.95, 44, 2.08, 51, 63], ['OpheliaKR', 'Twisted Fate', 55, 1.94, 52, 1.94, 51, 11], ['await0', 'Lee Sin', 48, 2.76, 54, 2.46, 53, 21]]
['taoyouw777', 'Evelynn', 52, 3.91, 52, 3.91, 52, 0]
['한 낳', 'Jayc

['쁘띠챌', 'Ezreal', 56, 2.64, 47, 2.57, 61, 55]
['aksrl', 'Fiddlesticks', 43, 6.17, 43, 6.17, 88, 0]
['렝가프로지망생', 'Rengar', 48, 2.58, 48, 2.58, 51, 0]
['mv point', 'Yone', 48, 2.7, 48, 2.7, 52, 0]
[['쁘 띠', 'Yuumi', 60, 5.99, 56, 4.71, 60, 186], ['쁘띠챌', 'Ezreal', 56, 2.64, 47, 2.57, 61, 55], ['aksrl', 'Fiddlesticks', 43, 6.17, 43, 6.17, 88, 0], ['렝가프로지망생', 'Rengar', 48, 2.58, 48, 2.58, 51, 0], ['mv point', 'Yone', 57, 2.29, 48, 2.7, 52, 54]]
['널 사랑하지 않아', 'Twitch', 80, 3.45, 56, 3.49, 59, 15]
['다이아2 20포인트', 'Taliyah', 50, 2.87, 50, 2.87, 53, 0]
['루나어린이', 'Lulu', 56, 3.3, 57, 3.59, 52, 55]
['PERFUM2', 'Camille', 53, 2.24, 44, 2.75, 53, 134]
['LNG JUG', 'Karthus', 56, 3.52, 56, 3.52, 55, 0]
[['널 사랑하지 않아', 'Twitch', 80, 3.45, 56, 3.49, 59, 15], ['다이아2 20포인트', 'Taliyah', 75, 2.43, 50, 2.87, 53, 8], ['루나어린이', 'Lulu', 56, 3.3, 57, 3.59, 52, 55], ['PERFUM2', 'Camille', 53, 2.24, 44, 2.75, 53, 134], ['LNG JUG', 'Karthus', 50, 1.63, 56, 3.52, 55, 2]]
['muzke', 'Elise', 79, 11.25, 79, 11.25, 53, 0]


['개스이야', 'Ezreal', 66, 3.35, 58, 3.49, 63, 236]
['sdifhgisdfdfs', 'Sylas', 52, 2.82, 46, 2.48, 49, 123]
['느금만수르', 'Lillia', 43, 3.27, 60, 3.67, 52, 21]
['asdsadasdsadasd', 'Jayce', 43, 1.44, 43, 1.44, 50, 0]
['The knight a', 'Pyke', 66, 2.88, 54, 2.97, 60, 121]
[['개스이야', 'Ezreal', 66, 3.35, 58, 3.49, 63, 236], ['sdifhgisdfdfs', 'Sylas', 52, 2.82, 46, 2.48, 49, 123], ['느금만수르', 'Lillia', 43, 3.27, 60, 3.67, 52, 21], ['asdsadasdsadasd', 'Jayce', 53, 1.98, 43, 1.44, 50, 287], ['The knight a', 'Pyke', 66, 2.88, 54, 2.97, 60, 121]]
['lv ren', 'Fiora', 50, 1.63, 50, 3.02, 53, 119]
['The knight a', 'Lee Sin', 33, 3.33, 54, 2.97, 60, 3]
['로빈슨이다', 'Jhin', 56, 4.14, 56, 4.14, 55, 0]
['솔랭즐겜아이디', 'Zoe', 46, 1.93, 46, 1.93, 56, 0]
['마음착한 아들님', 'Xerath', 55, 4.09, 54, 3.36, 59, 11]
[['lv ren', 'Fiora', 50, 1.63, 50, 3.02, 53, 119], ['The knight a', 'Lee Sin', 33, 3.33, 54, 2.97, 60, 3], ['로빈슨이다', 'Jhin', 50, 5.14, 56, 4.14, 55, 8], ['솔랭즐겜아이디', 'Zoe', 50, 2.0, 46, 1.93, 56, 2], ['마음착한 아들님', 'Xerath', 

In [120]:
len(dat_result)

698

In [121]:
dat_result[0]

{'gameId': 4709111969,
 'teamId': 200,
 'win': False,
 'participants': [['뱅 배준식', 'Camille', 50, 1.95, 57, 2.52, 51, 12],
  ['짜 룽', 'Lee Sin', 51, 3.56, 57, 2.5, 51, 632],
  ['happy   game ', 'Yone', 40, 1.62, 48, 1.34, 52, 10],
  ['잠 이 덜 깨다', 'Caitlyn', 57, 2.4, 44, 2.2, 51, 93],
  ['ha1 m1an bao bao', 'Taric', 61, 3.95, 50, 1.72, 54, 341]]}

In [122]:
non_list = []
for k in range(len(dat_result)):
    for j in dat['participants']:
        if j[-1] ==0:
            non_list.append(k)
            break
print(non_list)

[]


In [123]:
with open('Gameinfo_with_user_48336_to_60000_all_new.pickle', 'wb') as g:
    pickle.dump(dat_result, g)

In [131]:
dat_result_perfect = []
for k in range(len(dat_result)):
    if k not in non_list:
        dat_result_perfect.append(dat_result[k])
print(len(dat_result_perfect))

1181


In [132]:
with open('Gameinfo_with_user_70000_to_110000_new.pickle', 'wb') as f:
    pickle.dump(dat_result_perfect, f) #우선 완벽히 있는 애들울 넣고
    pickle.dump(dat_result, f) #그 다음에 결함 있는 애들을 넣자.

In [3]:
with open('Gameinfo_with_user_70000_to_110000_new.pickle', 'rb') as f:
    dat_result_perfect = pickle.load(f) #우선 완벽히 있는 애들울 넣고
    dat_result = pickle.load(f) #그 다음에 결함 있는 애들을 넣자.

In [7]:
match_timeline_list = [] #타임라인 데이터프레임을 모은 리스트
game_id_list = [] #게임ID 모은 리스트

In [8]:
#MAPDATA를 gameID에서 추출함.
for ind in tqdm(range(len(dat_result))): # 각 게임 아이디마다 요청
    game_id = dat_result[ind]['gameId']
    game_id_list.append(game_id)
    api_url = f'https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{game_id}?api_key={api_key}'
    r = requests.get(api_url)
    while r.status_code!=200: # 요청 제한 또는 오류로 인해 정상적으로 받아오지 않는 상태라면, 3초 간 시간을 지연
        time.sleep(3)
        r = requests.get(api_url)
    temp_match = pd.DataFrame(list(r.json().values())[0]) # 전체 데이터 저장 (데이터 값에 딕셔너리 형태로 필요한 정보가 저장)
    temp_timeline = pd.DataFrame()
    len_timeline = temp_match.shape[0]
    for i in range(len_timeline): # 각 게임의 타임라인이 모두 다르기 때문 (1분 가량마다 타임라인이 찍힌다)
        temp_current_timeline = pd.DataFrame(temp_match['participantFrames'].iloc[i]).T
        if i != (len_timeline-1):
            temp_position = pd.DataFrame(list(temp_current_timeline['position'].values), index=temp_current_timeline.index)
            temp_current_timeline = pd.concat([temp_current_timeline, temp_position], axis=1)
            temp_current_timeline.drop('position', axis=1, inplace=True)
        temp_current_timeline['timestamp'] = temp_match['timestamp'][i]
        temp_timeline = pd.concat([temp_timeline, temp_current_timeline], axis=0, sort=False)
    match_timeline_list.append(temp_timeline)

100%|██████████████████████████████████████████████████████████████████████████████| 1676/1676 [33:30<00:00,  1.20s/it]


In [10]:
#match_timeline_list - 확인 
match_timeline_list[0]

,participantId,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,dominionScore,teamScore,x,y,timestamp
1,3,500,500,1,0,0,0,0,0,560.0,581.0,0
2,5,500,500,1,0,0,0,0,0,560.0,361.0,0
3,4,500,500,1,0,0,0,0,0,351.0,293.0,0
4,1,500,500,1,0,0,0,0,0,221.0,471.0,0
5,2,500,500,1,0,0,0,0,0,311.0,649.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6,9,133,14243,16,15857,215,28,NaN,NaN,NaN,NaN,1980560
7,7,310,11493,15,13040,53,152,NaN,NaN,NaN,NaN,1980560
8,6,382,12482,15,13411,181,4,NaN,NaN,NaN,NaN,1980560
9,8,463,11838,15,14245,171,0,NaN,NaN,NaN,NaN,1980560


In [ ]:
for k in range(len(match_timeline_list)): #인덱스 강제정리
    cur_timeline = match_timeline_list[k]
    cur_timeline.sort_values(by=['timestamp', 'participantId'], axis=0, inplace=True)
    cur_timeline.index = list(range(1,11))*(len(cur_timeline)//10)

In [12]:
#match_timeline_list - 확인 
match_timeline_list[0]

,participantId,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,dominionScore,teamScore,x,y,timestamp
1,1,500,500,1,0,0,0,0,0,221.0,471.0,0
2,2,500,500,1,0,0,0,0,0,311.0,649.0,0
3,3,500,500,1,0,0,0,0,0,560.0,581.0,0
4,4,500,500,1,0,0,0,0,0,351.0,293.0,0
5,5,500,500,1,0,0,0,0,0,560.0,361.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6,6,382,12482,15,13411,181,4,NaN,NaN,NaN,NaN,1980560
7,7,310,11493,15,13040,53,152,NaN,NaN,NaN,NaN,1980560
8,8,463,11838,15,14245,171,0,NaN,NaN,NaN,NaN,1980560
9,9,133,14243,16,15857,215,28,NaN,NaN,NaN,NaN,1980560


In [13]:
#스펠 저장용
spell_api = 'http://ddragon.leagueoflegends.com/cdn/10.22.1/data/ko_KR/summoner.json'
r = requests.get(spell_api)
spell_info_df = pd.DataFrame(r.json())
spells = {}
for i in range(len(spell_info_df)):
    cur_spell = spell_info_df['data'].iloc[i]
    if 'CLASSIC' in cur_spell['modes']:
        spells[int(cur_spell['key'])] = cur_spell['name']
spells = sorted(spells.items(), key=lambda t : t[0])

In [14]:
#맵 좌표 스케일 조정용.

def MapScaler(data, x_range=(-120, 14870), y_range=(-120, 14980)): # x, y의 범위
    x = data['x'].astype('float64').values.reshape(-1, 1)
    y = data['y'].astype('float64').values.reshape(-1, 1)
    x_range = np.array(x_range).astype('float64').reshape(-1, 1)
    y_range = np.array(y_range).astype('float64').reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 512)) # 0~512로 변환
    scaler.fit(x_range)
    x = scaler.transform(x)
    scaler.fit(y_range)
    y = scaler.transform(y)
    return x, 512 - y

# 이미지는 왼쪽 위가 0인데, 지도는 왼쪽 아래가 0이다.
# 그래서 좌표만 뒤집어서 출력을 시키고 싶었는데, 찾기 힘들어서 임의로 y값을 뒤집었다.

In [15]:
#위의 매치데이터를 포함하는 게임인포 피클에서 가져옴.
with open('pickle/Gameinfo_70000_to_110000.pickle', 'rb') as f:
    match_df = pickle.load(f)

In [16]:
# 우선 인덱스 부여
match_df.index = range(len(match_df))

In [17]:
# 리스트에 있는 애들만 추려낸다. 
match_df0 = pd.DataFrame()
tmp = 0 #임시값
for i in dat_result:
    for j in range(tmp, len(match_df.index)):
        if match_df.iloc[j:j+1, :]['gameId'].values[0] == i['gameId']:  
            match_df0 = pd.concat((match_df0, match_df.iloc[j:j+1,:]))
            tmp = j
            break

In [18]:
# 길이 확인
len(match_df0), len(match_timeline_list)

(1676, 1676)

In [19]:
#수정된 방식 - 스마이트 보유자 <2면 단순히 정글미니언 킬수로 판단.

def SupJugPredict1(data, player_spells, i): # 서폿과 정글을 판단하기 위한 함수
    if i==1: # 블루팀
        final_timeline = data.iloc[-10:-5].copy()
        spells = player_spells[:5]
    if i==6: # 레드팀
        final_timeline = data.iloc[-5:].copy()
        spells = player_spells[5:10]
    smite_count = (spells == 11).sum()
    # 각 팀의 스마이트 개수 
    if smite_count <= 1: jungle_index = final_timeline['jungleMinionsKilled'].idxmax()
    elif  smite_count > 1: jungle_index = False
    # else: jungle_index = np.where(spells == 11)[0][0] + i # 강타를 들고있는 플레이어 번호
        
    #final_timeline = final_timeline.drop(index = np.where(spells == 11)[0][0]) #정글 제외
    final_timeline = final_timeline.drop(index=final_timeline['jungleMinionsKilled'].idxmax()) 
    return final_timeline['minionsKilled'].idxmin(), jungle_index # 정글을 제외하고 cs가 가장 적은 사람이 서포터

def LanePredict1(data, support_bool=False, jungle_bool=True):
    lane = {'TOP': 0, 'MID': 0, 'BOT_CARRY': 0, 'JUNGLE':0}
    etc = 0
    for xi, yi in zip(data['x'], data['y']):
        if (xi > 20) & (xi < 60) & (yi > 30) & (yi < 300): lane['TOP'] += 1
        elif (xi > 20) & (xi < 150) & (yi > 20) & (yi < 160) & (1-(xi>90) | 1-(yi>120)): lane['TOP'] += 1
        elif (xi > 30) & (xi < 300) & (yi > 20) & (yi < 60): lane['TOP'] += 1
        elif (xi > 195) & (xi < 265) & (yi > 250) & (yi < 310): lane['MID'] += 1
        elif (xi > 220) & (xi < 295) & (yi > 220) & (yi < 290): lane['MID'] += 1
        elif (xi > 250) & (xi < 320) & (yi > 200) & (yi < 260): lane['MID'] += 1
        elif (xi > 290) & (xi < 350) & (yi > 160) & (yi < 215): lane['MID'] += 1
        elif (xi > 160) & (xi < 220) & (yi > 290) & (yi < 340): lane['MID'] += 1
        elif (xi > 220) & (xi < 460) & (yi > 435) & (yi < 485): lane['BOT_CARRY'] += 1
        elif (xi > 400) & (xi < 490) & (yi > 385) & (yi < 480): lane['BOT_CARRY'] += 1
        elif (xi > 440) & (xi < 490) & (yi > 220) & (yi < 455): lane['BOT_CARRY'] += 1
        elif (xi > 0) & (xi < 170) & (yi > 340) & (yi < 512): etc += 1
        elif (xi > 340) & (xi < 512) & (yi > 0) & (yi < 170): etc += 1
        else: lane['JUNGLE'] += 1
    if jungle_bool:
        #if max(lane, key=lane.get) == 'JUNGLE': return 'JUNGLE', lane #정글이면 확정.
        #elif lane.get('JUNGLE') >4 :  return 'JUNGLE', lane #정글에서 5개 이상 찍히면 확정.
        del lane['JUNGLE']
    # 예측된 서포터 번호가 봇에 가장 오래 있었으면 서포터로 확정, 아니면 라인으로 판단
    if support_bool & any([max(lane, key=lane.get) == 'BOT_CARRY' ]): return 'BOT_SUPPORT', lane
        
    return max(lane, key=lane.get), lane

In [20]:
# 라인 계산 - 순수 포지션 방식으로 재계산

lane_calculated_new = pd.DataFrame()
lane_smite_1_each_new = pd.DataFrame()
lane_from_data_new = pd.DataFrame()
for k in range(len(match_timeline_list)):
#    if k!=45:
        cur_timeline = match_timeline_list[k].copy()
        cur_timeline['jungleMinionsKilled'] = cur_timeline['jungleMinionsKilled'].astype('float64')
        cur_timeline['minionsKilled'] = cur_timeline['minionsKilled'].astype('float64')
        # 타임스탬프는 op.gg가 나타내는 아이템 타임스탬프와 비교 결과, 타임스탬프 값의 1000 단위가 1초인 것을 파악함
        cur_timeline['timestamp'] = cur_timeline['timestamp'] / (1000*60)  # 타임스탬프 값을 분 단위로 변환
        condition = (cur_timeline['timestamp'] > 2) & (cur_timeline['timestamp'] < min(match_df0.iloc[k, :]['gameDuration']/60 -2, 16) )  # 2분 이하 : 미니언이 라인 도착 전, 시야 확보와 정글 리쉬
                        # 16분 이상 또는 게임 끝나기 2분전... : 라인전을 끝내고 다른 라인으로 이동할 수 있음
        cur_timeline = cur_timeline.loc[condition].copy()
        cur_timeline['x'], cur_timeline['y'] = MapScaler(cur_timeline)
        player_spells = [(data['spell1Id'], data['spell2Id']) for data in match_df0.iloc[k, :]['participants'] ] # 스펠 확인
        player_spells = np.array(player_spells)

        lane = {}
        smite_exception = False # 강타 갯수가 각 팀에 하나씩만 있게 확인
        role_temp = [[],[]]

        lane_data = {}
        for i in range(10): #데이터프레임에서 계산...
            info = match_df0['participants'].iloc[k][i]['timeline']['role'], match_df0['participants'].iloc[k][i]['timeline']['lane']
            lane_data[f'lane{i+1}'] = info
        lane_data['gameId'] = match_df0.iloc[k, :]['gameId']


        for i in range(1, 11, 5):  # 라인 계산
            support_index, jungle_index = SupJugPredict1(cur_timeline, player_spells, i)
            smite_exception = smite_exception or (not(jungle_index))
            #smite_index = np.where(spells == 11)[0][0] + i 

            for j in range(i, i+5):
                if j == jungle_index: # 우선 정글 인덱스가 존재해도 정글에서 있는 시점이 없으면.... 
                    lane[f'lane{j}'] = 'JUNGLE'
                    continue
                cur_player = cur_timeline.loc[j].copy()
                role, dic_lane = LanePredict1(cur_player, support_index==j, jungle_index)
                if role == 'BOT_SUPPORT': 
                    lane[f'lane{j}'] = 'BOT_SUPPORT'
                    role_temp[i//5].append('BOT_SUPPORT')
                elif sorted(dic_lane.values())[-1] - sorted(dic_lane.values())[-2] >2: 
                    lane[f'lane{j}'] = role
                    role_temp[i//5].append(role)
                elif sum(dic_lane.values())-dic_lane[role] <=2 : 
                    lane[f'lane{j}'] = role
                    role_temp[i//5].append(role)
                else: 
                    lane[f'lane{j}'] = dic_lane #분명하지 않으면 조건 조작

        for i in range(1,11,5):
            for j in range(i,i+5):
                if str(type(lane[f'lane{j}'])) == "<class 'dict'>":
                    val = lane[f'lane{j}']
                    role = max(val, key=val.get)
                    if role not in role_temp[i//5]:
                        lane[f'lane{j}'] = role
                    else:
                        lane[f'lane{j}'] = 'UNDECIDED'


        lane['gameId'] = match_df0.iloc[k, :]['gameId']
        lane['smiteException'] = smite_exception
        lane['codeIndex'] =k
        lane_data['smiteException'] = smite_exception
        lane_data['codeIndex'] =k
        lane = pd.Series(lane, name = match_df0.index[k])
        lane_data = pd.Series(lane_data, name = match_df0.index[k])
        if any([(lane.value_counts().get('UNDECIDED', 0)> 4), (lane[0:5].value_counts().max()>2) , (lane[5:10].value_counts().max()>2)]): # 각 팀에 한 라인이 3명 이상이면 계산 착오로 판단하여 데이터 삭제
            print(f"{k}번째 계산이 잘못되었습니다.")
            continue
        if len(lane_calculated_new) == 0:
            lane_calculated_new = pd.concat([lane_calculated_new, lane], axis=1, sort=False)
            lane_smite_1_each_new = pd.concat([lane_smite_1_each_new, lane], axis=1, sort=False)
            lane_from_data_new = pd.concat([lane_from_data_new, lane_data], axis=1, sort=False)
        elif lane['gameId'] not in list(lane_calculated_new.loc['gameId', :]):
            if match_df0.iloc[k, :]['gameDuration']<600: # 플레이타임 10분 이상만 추리기
                print(f'오류 : {k}번째 게임은 짧음.')
            else:
                lane_calculated_new = pd.concat([lane_calculated_new, lane], axis=1, sort=False)
                lane_from_data_new = pd.concat([lane_from_data_new, lane_data], axis=1, sort=False)
                if not lane['smiteException']: 
                    lane_smite_1_each_new = pd.concat([lane_smite_1_each_new, lane], axis=1, sort=False)


오류 : 27번째 게임은 짧음.
490번째 계산이 잘못되었습니다.
1305번째 계산이 잘못되었습니다.


In [21]:
# 수정된 알고리즘으로 계산
lane_calculated_new

,3,11,25,35,47,52,65,81,93,103,...,22734,22737,22741,22745,22764,22768,22802,22826,22837,22889
lane1,BOT_CARRY,BOT_SUPPORT,BOT_SUPPORT,TOP,MID,BOT_SUPPORT,BOT_SUPPORT,TOP,JUNGLE,BOT_SUPPORT,...,BOT_SUPPORT,MID,BOT_CARRY,BOT_SUPPORT,JUNGLE,BOT_SUPPORT,MID,TOP,MID,JUNGLE
lane2,BOT_SUPPORT,TOP,BOT_CARRY,BOT_SUPPORT,BOT_CARRY,TOP,JUNGLE,BOT_SUPPORT,BOT_SUPPORT,JUNGLE,...,TOP,TOP,BOT_SUPPORT,BOT_CARRY,BOT_SUPPORT,BOT_CARRY,JUNGLE,BOT_CARRY,TOP,MID
lane3,TOP,JUNGLE,JUNGLE,BOT_CARRY,BOT_SUPPORT,MID,BOT_CARRY,JUNGLE,MID,BOT_CARRY,...,MID,JUNGLE,TOP,MID,BOT_CARRY,MID,BOT_CARRY,JUNGLE,BOT_CARRY,BOT_CARRY
lane4,MID,BOT_CARRY,MID,JUNGLE,JUNGLE,JUNGLE,TOP,BOT_CARRY,TOP,MID,...,BOT_CARRY,BOT_SUPPORT,MID,JUNGLE,TOP,JUNGLE,BOT_SUPPORT,MID,BOT_SUPPORT,BOT_SUPPORT
lane5,JUNGLE,MID,TOP,MID,TOP,BOT_CARRY,MID,MID,BOT_CARRY,TOP,...,JUNGLE,BOT_CARRY,JUNGLE,TOP,MID,TOP,TOP,BOT_SUPPORT,JUNGLE,TOP
lane6,MID,BOT_CARRY,BOT_CARRY,MID,BOT_SUPPORT,TOP,JUNGLE,JUNGLE,TOP,MID,...,BOT_CARRY,JUNGLE,BOT_CARRY,BOT_SUPPORT,BOT_SUPPORT,BOT_CARRY,MID,MID,TOP,MID
lane7,JUNGLE,BOT_SUPPORT,JUNGLE,BOT_CARRY,TOP,BOT_CARRY,MID,TOP,BOT_SUPPORT,BOT_CARRY,...,JUNGLE,MID,JUNGLE,TOP,JUNGLE,JUNGLE,JUNGLE,BOT_SUPPORT,MID,TOP
lane8,BOT_CARRY,JUNGLE,MID,JUNGLE,BOT_CARRY,BOT_SUPPORT,TOP,BOT_SUPPORT,BOT_CARRY,TOP,...,MID,TOP,MID,JUNGLE,MID,TOP,BOT_SUPPORT,BOT_CARRY,BOT_CARRY,JUNGLE
lane9,TOP,TOP,BOT_SUPPORT,TOP,MID,JUNGLE,BOT_CARRY,MID,JUNGLE,JUNGLE,...,TOP,BOT_CARRY,BOT_SUPPORT,BOT_CARRY,BOT_CARRY,BOT_SUPPORT,BOT_CARRY,JUNGLE,JUNGLE,BOT_CARRY
lane10,BOT_SUPPORT,MID,TOP,BOT_SUPPORT,JUNGLE,MID,BOT_SUPPORT,BOT_CARRY,MID,BOT_SUPPORT,...,BOT_SUPPORT,BOT_SUPPORT,TOP,MID,TOP,MID,TOP,TOP,BOT_SUPPORT,BOT_SUPPORT


In [22]:
# 수정된 알고리즘으로 계산 - 원 피클에서 추출
lane_from_data_new

,3,11,25,35,47,52,65,81,93,103,...,22734,22737,22741,22745,22764,22768,22802,22826,22837,22889
lane1,"(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, NONE)","(SOLO, TOP)","(SOLO, MIDDLE)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, NONE)","(NONE, JUNGLE)","(DUO_SUPPORT, BOTTOM)",...,"(DUO_SUPPORT, BOTTOM)","(DUO, MIDDLE)","(SOLO, BOTTOM)","(DUO_SUPPORT, MIDDLE)","(NONE, JUNGLE)","(DUO_SUPPORT, NONE)","(DUO, TOP)","(DUO, TOP)","(SOLO, MIDDLE)","(NONE, JUNGLE)"
lane2,"(DUO_SUPPORT, BOTTOM)","(DUO, TOP)","(DUO_SUPPORT, NONE)","(DUO_SUPPORT, BOTTOM)","(DUO_CARRY, BOTTOM)","(SOLO, TOP)","(NONE, JUNGLE)","(DUO_SUPPORT, NONE)","(DUO_SUPPORT, BOTTOM)","(NONE, JUNGLE)",...,"(SOLO, TOP)","(DUO, MIDDLE)","(DUO_SUPPORT, MIDDLE)","(SOLO, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, NONE)","(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)","(SOLO, TOP)","(DUO, TOP)"
lane3,"(DUO, MIDDLE)","(NONE, JUNGLE)","(DUO_SUPPORT, NONE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(SOLO, MIDDLE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, NONE)","(SOLO, MIDDLE)","(DUO, BOTTOM)",...,"(SOLO, MIDDLE)","(NONE, JUNGLE)","(SOLO, TOP)","(DUO_CARRY, MIDDLE)","(DUO_CARRY, BOTTOM)","(DUO, NONE)","(DUO_CARRY, BOTTOM)","(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)","(DUO_CARRY, BOTTOM)"
lane4,"(DUO, MIDDLE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, NONE)","(NONE, JUNGLE)","(NONE, JUNGLE)","(NONE, JUNGLE)","(SOLO, TOP)","(DUO_SUPPORT, NONE)","(SOLO, TOP)","(DUO, BOTTOM)",...,"(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_CARRY, MIDDLE)","(NONE, JUNGLE)","(SOLO, TOP)","(DUO_SUPPORT, NONE)","(DUO_SUPPORT, BOTTOM)","(DUO, TOP)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, BOTTOM)"
lane5,"(NONE, JUNGLE)","(DUO, TOP)","(DUO_SUPPORT, NONE)","(SOLO, MIDDLE)","(SOLO, TOP)","(DUO_CARRY, BOTTOM)","(SOLO, MIDDLE)","(DUO_SUPPORT, NONE)","(DUO_CARRY, BOTTOM)","(SOLO, TOP)",...,"(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)","(NONE, JUNGLE)","(SOLO, TOP)","(SOLO, MIDDLE)","(DUO_SUPPORT, NONE)","(DUO, TOP)","(DUO_SUPPORT, BOTTOM)","(NONE, JUNGLE)","(DUO, TOP)"
lane6,"(SOLO, MIDDLE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, NONE)","(SOLO, MIDDLE)","(DUO_SUPPORT, BOTTOM)","(SOLO, TOP)","(NONE, JUNGLE)","(DUO_SUPPORT, NONE)","(SOLO, TOP)","(SOLO, MIDDLE)",...,"(DUO_CARRY, BOTTOM)","(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, NONE)","(SOLO, MIDDLE)","(SOLO, MIDDLE)","(SOLO, TOP)","(DUO, TOP)"
lane7,"(DUO_SUPPORT, TOP)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, NONE)","(DUO_CARRY, BOTTOM)","(SOLO, TOP)","(DUO_CARRY, BOTTOM)","(SOLO, MIDDLE)","(DUO_CARRY, NONE)","(DUO_SUPPORT, BOTTOM)","(DUO_CARRY, BOTTOM)",...,"(NONE, JUNGLE)","(DUO, TOP)","(NONE, JUNGLE)","(SOLO, TOP)","(NONE, JUNGLE)","(DUO_SUPPORT, NONE)","(NONE, JUNGLE)","(DUO_SUPPORT, BOTTOM)","(SOLO, MIDDLE)","(DUO, TOP)"
lane8,"(DUO_CARRY, BOTTOM)","(NONE, JUNGLE)","(DUO_SUPPORT, NONE)","(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(SOLO, TOP)","(DUO_SUPPORT, NONE)","(DUO_CARRY, BOTTOM)","(SOLO, TOP)",...,"(SOLO, MIDDLE)","(DUO, TOP)","(SOLO, MIDDLE)","(NONE, JUNGLE)","(SOLO, MIDDLE)","(DUO, NONE)","(DUO_SUPPORT, BOTTOM)","(DUO_CARRY, BOTTOM)","(DUO_CARRY, BOTTOM)","(NONE, JUNGLE)"
lane9,"(DUO_CARRY, TOP)","(SOLO, TOP)","(DUO_SUPPORT, NONE)","(SOLO, TOP)","(SOLO, MIDDLE)","(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, NONE)","(NONE, JUNGLE)","(NONE, JUNGLE)",...,"(SOLO, TOP)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(DUO_CARRY, BOTTOM)","(DUO_CARRY, BOTTOM)","(DUO_SUPPORT, NONE)","(DUO_CARRY, BOTTOM)","(NONE, JUNGLE)","(NONE, JUNGLE)","(DUO_CARRY, BOTTOM)"
lane10,"(DUO_SUPPORT, BOTTOM)","(SOLO, MIDDLE)","(DUO, NONE)","(DUO_SUPPORT, BOTTOM)","(NONE, JUNGLE)","(SOLO, MIDDLE)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, NONE)","(SOLO, MIDDLE)","(DUO_SUPPORT, BOTTOM)",...,"(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, BOTTOM)","(SOLO, TOP)","(SOLO, MIDDLE)","(SOLO, TOP)","(DUO_SUPPORT, NONE)","(SOLO, TOP)","(SOLO, TOP)","(DUO_SUPPORT, BOTTOM)","(DUO_SUPPORT, BOTTOM)"


In [23]:
# 중복 포지션 삭제.
jr_drop_new = []
for jr in lane_calculated_new.columns:
    if any([(lane_calculated_new[jr][0:5].value_counts().max()>1) , (lane_calculated_new[jr][5:10].value_counts().max()>1), (lane_calculated_new[jr].value_counts().get('UNDECIDED'))]):
        jr_drop_new.append(jr)

In [24]:
len(jr_drop_new)

49

In [25]:
lane_calculated_new.loc[:, jr_drop_new] #예외처리용 출력

,814,1043,1134,1659,1923,1943,2527,2954,3784,3914,...,15321,15467,17068,20218,21371,21405,21589,22248,22264,22438
lane1,BOT_CARRY,MID,BOT_CARRY,BOT_CARRY,BOT_CARRY,TOP,BOT_SUPPORT,TOP,BOT_CARRY,UNDECIDED,...,MID,TOP,MID,MID,MID,MID,BOT_CARRY,TOP,MID,JUNGLE
lane2,BOT_SUPPORT,UNDECIDED,MID,TOP,JUNGLE,UNDECIDED,MID,JUNGLE,BOT_CARRY,JUNGLE,...,TOP,MID,TOP,JUNGLE,BOT_CARRY,MID,MID,MID,TOP,BOT_SUPPORT
lane3,TOP,JUNGLE,BOT_SUPPORT,UNDECIDED,MID,JUNGLE,TOP,MID,MID,BOT_CARRY,...,JUNGLE,MID,UNDECIDED,BOT_CARRY,TOP,JUNGLE,BOT_SUPPORT,MID,JUNGLE,TOP
lane4,JUNGLE,TOP,JUNGLE,MID,BOT_SUPPORT,BOT_CARRY,JUNGLE,UNDECIDED,JUNGLE,BOT_SUPPORT,...,BOT_CARRY,BOT_CARRY,BOT_CARRY,BOT_SUPPORT,BOT_CARRY,TOP,TOP,JUNGLE,UNDECIDED,MID
lane5,MID,TOP,TOP,JUNGLE,TOP,BOT_CARRY,BOT_CARRY,BOT_SUPPORT,TOP,TOP,...,BOT_SUPPORT,JUNGLE,JUNGLE,TOP,JUNGLE,BOT_CARRY,JUNGLE,BOT_CARRY,BOT_CARRY,TOP
lane6,JUNGLE,TOP,TOP,BOT_CARRY,UNDECIDED,MID,TOP,JUNGLE,BOT_SUPPORT,BOT_CARRY,...,TOP,MID,TOP,JUNGLE,BOT_CARRY,BOT_SUPPORT,JUNGLE,JUNGLE,MID,JUNGLE
lane7,BOT_SUPPORT,BOT_CARRY,MID,MID,BOT_CARRY,BOT_CARRY,UNDECIDED,BOT_SUPPORT,BOT_CARRY,BOT_SUPPORT,...,BOT_CARRY,BOT_CARRY,JUNGLE,TOP,BOT_SUPPORT,MID,BOT_CARRY,MID,JUNGLE,BOT_SUPPORT
lane8,MID,BOT_CARRY,BOT_CARRY,JUNGLE,JUNGLE,TOP,MID,TOP,JUNGLE,JUNGLE,...,JUNGLE,JUNGLE,MID,MID,TOP,JUNGLE,BOT_SUPPORT,MID,MID,BOT_CARRY
lane9,TOP,TOP,JUNGLE,TOP,MID,BOT_SUPPORT,BOT_CARRY,BOT_CARRY,TOP,TOP,...,BOT_CARRY,BOT_SUPPORT,BOT_SUPPORT,BOT_SUPPORT,JUNGLE,BOT_CARRY,TOP,UNDECIDED,UNDECIDED,TOP
lane10,UNDECIDED,JUNGLE,MID,BOT_SUPPORT,TOP,JUNGLE,JUNGLE,MID,MID,MID,...,TOP,TOP,BOT_CARRY,TOP,MID,TOP,TOP,TOP,TOP,MID


In [26]:
lane_calculated_new_0 = lane_calculated_new.drop(columns=jr_drop_new, axis=1)
lane_from_data_new_0 = lane_from_data_new.drop(columns=jr_drop_new, axis=1)

In [28]:
#피클로 데이터 저장
with open('position_70000_to_110000.pickle', 'wb') as f:
    pickle.dump(lane_calculated_new,f)
    pickle.dump(lane_calculated_new_0, f)
    

In [29]:
with open('mapdata_70000_to_110000.pickle', 'wb') as g:
    pickle.dump(match_timeline_list, g)

In [30]:
len(dat_result_perfect)

1181

In [18]:
dat_result_with_pos = []
for dat in dat_result_perfect: #완벽히 포지션이 있는 애들만 확인.
    dat0 = dat.copy() #데이터 복제분
    for game_id in lane_calculated_new_0.loc['gameId']: #게임아이디로 동일성 확인
        if dat0['gameId'] == game_id:
            dat_team = []
            if dat0['teamId'] == 100:
                for l in range(5):
                    dat_team.append(lane_calculated_new_0.loc[f'lane{l+1}', lane_calculated_new_0.loc['gameId']==game_id].values[0])
            elif dat0['teamId'] == 200:
                for l in range(5):
                    dat_team.append(lane_calculated_new_0.loc[f'lane{l+6}', lane_calculated_new_0.loc['gameId']==game_id].values[0])
            if set(dat_team) == {'TOP', 'MID', 'BOT_CARRY', 'BOT_SUPPORT', 'JUNGLE'}: #한명씩 있을 때에만...
                for l in range(5):
                    dat0['participants'][l].insert(2, dat_team[l])
                dat_result_with_pos.append(dat0)
            break
            

In [19]:
dat_result_with_pos[0]

{'gameId': 4751789156,
 'teamId': 100,
 'win': True,
 'participants': [['책임을지다', 'Ashe', 'BOT_CARRY', 56, 2.96, 38, 1.82, 55, 151],
  ['paranoicl', 'Senna', 'BOT_SUPPORT', 70, 5.32, 56, 5.5, 63, 10],
  ['KILLBURN', 'Maokai', 'TOP', 49, 2.54, 33, 1.38, 58, 43],
  ['AtheartOnlyme ', 'Kassadin', 'MID', 40, 2.18, 52, 2.75, 53, 20],
  ['Tuzai chang', "Rek'Sai", 'JUNGLE', 55, 3.38, 54, 3.76, 52, 321]]}

In [20]:
cnt =0
for dat in dat_result_with_pos:
    dat_res = [dat['participants'][l][2] for l in range(5)]
    if set(dat_res)!={'TOP', 'MID', 'BOT_CARRY', 'BOT_SUPPORT', 'JUNGLE'}:
        cnt +=1
print(cnt)

0


In [22]:
len(dat_result_with_pos)

1143

In [23]:
with open('Gameinfo_70000_to_110000_with_user_and_position_mod.pickle', 'wb') as f:
    pickle.dump(dat_result_with_pos, f)